# Start

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import time
import pickle
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
import tensorflow as tf

In [ ]:
def add_result(cfg, model_history):
    file_name = '/content/drive/MyDrive/DS/chest-xray-pneumonia/chest-xray-MobileNetV2.pkl'

    with open(file_name, 'rb') as f:
        result = pickle.load(f)

    result[cfg] = model_history

    with open(file_name, 'wb') as f:
        pickle.dump(result, f)

In [ ]:
!unzip -q /content/drive/MyDrive/DS/chest-xray-pneumonia/chest-xray-pneumonia.zip -d /content
base_path = '/content/chest_xray'

print(os.listdir(base_path))

!rm -r '/content/chest_xray/__MACOSX'
!rm -r '/content/chest_xray/chest_xray'

replace /content/chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
['train', 'test', '__MACOSX', 'val', 'chest_xray']


In [ ]:
print(os.listdir(base_path))

total_files = 0
ext_list = []

for root, dirs, files in os.walk(base_path):
    total_files += len(files)
    level = root.replace(base_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}- {os.path.basename(root)}/ ({len(files)})")
    for file in files:
        if file == '.DS_Store':
            file_path = os.path.join(root, file)
            os.remove(file_path)
        else:
            name, ext = os.path.splitext(file)
            if ext not in ext_list:
                ext_list.append(ext)

print('\nВсего файлов:', total_files)
print('Расширения файлов:', ext_list)

['train', 'test', 'val']
- chest_xray/ (0)
    - train/ (0)
        - NORMAL/ (1341)
        - PNEUMONIA/ (3875)
    - test/ (0)
        - NORMAL/ (234)
        - PNEUMONIA/ (390)
    - val/ (0)
        - NORMAL/ (8)
        - PNEUMONIA/ (8)

Всего файлов: 5856
Расширения файлов: ['.jpeg']


In [ ]:
train_dir = os.path.join(base_path, 'train')
val_dir   = os.path.join(base_path, 'test')
test_dir  = os.path.join(base_path, 'val')

# cfg 1.0 - base_model
* default learning_rate=0.001

In [ ]:
def model_train():
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train()

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 136s 754ms/step - AUC: 0.9314 - Precision: 0.9070 - Recall: 0.9298 - accuracy: 0.8775 - loss: 0.2698 - val_AUC: 0.9690 - val_Precision: 0.8151 - val_Recall: 0.9949 - val_accuracy: 0.8558 - val_loss: 0.3457
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 114s 698ms/step - AUC: 0.9860 - Precision: 0.9648 - Recall: 0.9660 - accuracy: 0.9486 - loss: 0.1325 - val_AUC: 0.9656 - val_Precision: 0.8109 - val_Recall: 0.9897 - val_accuracy: 0.8494 - val_loss: 0.3859
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 114s 700ms/step - AUC: 0.9865 - Precision: 0.9641 - Recall: 0.9655 - accuracy: 0.9478 - loss: 0.1289 - val_AUC: 0.9701 - val_Precision: 0.8710 - val_Recall: 0.9692 - val_accuracy: 0.8910 - val_loss: 0.2477
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 694ms/step - AUC: 0.9882 - Precision: 0.9660 - Recall: 0.9706 - accuracy: 0.9527 - loss: 0.1172 - val_AUC: 0.9656 - val_Precision: 0.8326 - val_Recall: 0.9821 - val_accuracy: 0.8654 - val_loss: 0.3521
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_1.0'
add_result(cfg, model_history)

In [ ]:
model.save('/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-base.keras')

## cfg 1.1
* learning_rate=<font color=orange>0.0001</font>



In [ ]:
def model_train():
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train()

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 130s 738ms/step - AUC: 0.7733 - Precision: 0.8174 - Recall: 0.8529 - accuracy: 0.7497 - loss: 0.4968 - val_AUC: 0.9383 - val_Precision: 0.7631 - val_Recall: 0.9744 - val_accuracy: 0.7949 - val_loss: 0.3943
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 695ms/step - AUC: 0.9625 - Precision: 0.9316 - Recall: 0.9458 - accuracy: 0.9090 - loss: 0.2213 - val_AUC: 0.9533 - val_Precision: 0.7946 - val_Recall: 0.9821 - val_accuracy: 0.8301 - val_loss: 0.3730
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 116s 713ms/step - AUC: 0.9786 - Precision: 0.9526 - Recall: 0.9604 - accuracy: 0.9353 - loss: 0.1679 - val_AUC: 0.9591 - val_Precision: 0.8468 - val_Recall: 0.9641 - val_accuracy: 0.8686 - val_loss: 0.2877
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 114s 702ms/step - AUC: 0.9842 - Precision: 0.9631 - Recall: 0.9635 - accuracy: 0.9456 - loss: 0.1448 - val_AUC: 0.9620 - val_Precision: 0.8243 - val_Recall: 0.9744 - val_accuracy: 0.8542 - val_loss: 0.3120
Epoch 5/10
163/

In [ ]:
cfg = 'cfg_1.1'
add_result(cfg, model_history)

In [ ]:
model.save('/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-best.keras')

## cfg 1.2
* learning_rate=<font color=orange>0.00005</font>

In [ ]:
def model_train(learning_rate):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 129s 732ms/step - AUC: 0.6488 - Precision: 0.7676 - Recall: 0.7338 - accuracy: 0.6467 - loss: 0.6661 - val_AUC: 0.9199 - val_Precision: 0.7475 - val_Recall: 0.9641 - val_accuracy: 0.7740 - val_loss: 0.4278
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 695ms/step - AUC: 0.9531 - Precision: 0.9173 - Recall: 0.9644 - accuracy: 0.9090 - loss: 0.2557 - val_AUC: 0.9484 - val_Precision: 0.7796 - val_Recall: 0.9795 - val_accuracy: 0.8141 - val_loss: 0.3739
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 112s 690ms/step - AUC: 0.9672 - Precision: 0.9348 - Recall: 0.9496 - accuracy: 0.9130 - loss: 0.2105 - val_AUC: 0.9590 - val_Precision: 0.8219 - val_Recall: 0.9821 - val_accuracy: 0.8558 - val_loss: 0.3250
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 694ms/step - AUC: 0.9778 - Precision: 0.9451 - Recall: 0.9600 - accuracy: 0.9282 - loss: 0.1738 - val_AUC: 0.9632 - val_Precision: 0.8170 - val_Recall: 0.9846 - val_accuracy: 0.8526 - val_loss: 0.3268
Epoch 5/10
163/

In [ ]:
cfg = 'cfg_1.2'
add_result(cfg, model_history)

In [ ]:
model.save('/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-best.keras')

# cfg 2.0
* learning_rate=0.00005
* <font color=orange>class_weights</font>



In [ ]:
def model_train(learning_rate):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 128s 726ms/step - AUC: 0.6853 - Precision: 0.8366 - Recall: 0.6340 - accuracy: 0.6362 - loss: 0.6856 - val_AUC: 0.9108 - val_Precision: 0.8255 - val_Recall: 0.8974 - val_accuracy: 0.8173 - val_loss: 0.3854
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 114s 697ms/step - AUC: 0.9452 - Precision: 0.9553 - Recall: 0.8719 - accuracy: 0.8735 - loss: 0.3004 - val_AUC: 0.9351 - val_Precision: 0.8747 - val_Recall: 0.9128 - val_accuracy: 0.8638 - val_loss: 0.3247
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 112s 689ms/step - AUC: 0.9698 - Precision: 0.9731 - Recall: 0.8922 - accuracy: 0.9025 - loss: 0.2257 - val_AUC: 0.9469 - val_Precision: 0.8756 - val_Recall: 0.9205 - val_accuracy: 0.8686 - val_loss: 0.2951
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 112s 689ms/step - AUC: 0.9761 - Precision: 0.9780 - Recall: 0.9023 - accuracy: 0.9133 - loss: 0.2015 - val_AUC: 0.9543 - val_Precision: 0.8555 - val_Recall: 0.9410 - val_accuracy: 0.8638 - val_loss: 0.2871
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_2.0'
add_result(cfg, model_history)

In [ ]:
model.save('/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-best.keras')

# cfg 3.0
* learning_rate=0.00005
* class_weights
* base_model.trainable = <font color=orange>True</font>

In [ ]:
def model_train(learning_rate):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = True  #   [!]

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 2,387,969 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 180s 768ms/step - AUC: 0.9094 - Precision: 0.9393 - Recall: 0.8049 - accuracy: 0.8224 - loss: 0.3513 - val_AUC: 0.8281 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 1.2737
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 122s 750ms/step - AUC: 0.9917 - Precision: 0.9839 - Recall: 0.9562 - accuracy: 0.9563 - loss: 0.1175 - val_AUC: 0.8248 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 1.7189
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 723ms/step - AUC: 0.9932 - Precision: 0.9885 - Recall: 0.9594 - accuracy: 0.9617 - loss: 0.0992 - val_AUC: 0.7495 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 1.9811
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 722ms/step - AUC: 0.9971 - Precision: 0.9943 - Recall: 0.9736 - accuracy: 0.9763 - loss: 0.0631 - val_AUC: 0.7261 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 2.1340
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_3.0'
add_result(cfg, model_history)

# cfg 4.0
* learning_rate=0.00005
* class_weights
* horizontal_flip=<font color=orange>True</font>

In [ ]:
def model_train(learning_rate):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 138s 772ms/step - AUC: 0.8224 - Precision: 0.8846 - Recall: 0.7688 - accuracy: 0.7515 - loss: 0.5028 - val_AUC: 0.9271 - val_Precision: 0.8853 - val_Recall: 0.8513 - val_accuracy: 0.8381 - val_loss: 0.3568
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 116s 712ms/step - AUC: 0.9581 - Precision: 0.9689 - Recall: 0.8693 - accuracy: 0.8820 - loss: 0.2701 - val_AUC: 0.9445 - val_Precision: 0.8864 - val_Recall: 0.9000 - val_accuracy: 0.8654 - val_loss: 0.3039
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 115s 706ms/step - AUC: 0.9751 - Precision: 0.9794 - Recall: 0.9013 - accuracy: 0.9131 - loss: 0.2103 - val_AUC: 0.9537 - val_Precision: 0.8821 - val_Recall: 0.9205 - val_accuracy: 0.8734 - val_loss: 0.2779
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 716ms/step - AUC: 0.9774 - Precision: 0.9757 - Recall: 0.9176 - accuracy: 0.9222 - loss: 0.1979 - val_AUC: 0.9562 - val_Precision: 0.8780 - val_Recall: 0.9231 - val_accuracy: 0.8718 - val_loss: 0.2697
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_4.0'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 4.1
* learning_rate=0.00005
* class_weights
* vertical_flip=<font color=orange>True</font>

In [ ]:
def model_train(learning_rate):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        vertical_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 130s 722ms/step - AUC: 0.7100 - Precision: 0.8522 - Recall: 0.6252 - accuracy: 0.6460 - loss: 0.6294 - val_AUC: 0.9184 - val_Precision: 0.8962 - val_Recall: 0.8410 - val_accuracy: 0.8397 - val_loss: 0.3884
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 719ms/step - AUC: 0.9469 - Precision: 0.9584 - Recall: 0.8564 - accuracy: 0.8675 - loss: 0.3091 - val_AUC: 0.9357 - val_Precision: 0.9200 - val_Recall: 0.8846 - val_accuracy: 0.8798 - val_loss: 0.3292
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 110s 672ms/step - AUC: 0.9645 - Precision: 0.9661 - Recall: 0.8898 - accuracy: 0.8960 - loss: 0.2466 - val_AUC: 0.9450 - val_Precision: 0.9255 - val_Recall: 0.8923 - val_accuracy: 0.8878 - val_loss: 0.3003
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 141s 669ms/step - AUC: 0.9713 - Precision: 0.9686 - Recall: 0.8983 - accuracy: 0.9041 - loss: 0.2199 - val_AUC: 0.9506 - val_Precision: 0.8881 - val_Recall: 0.9154 - val_accuracy: 0.8750 - val_loss: 0.2832
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_4.1'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 4.2
* learning_rate=0.00005
* class_weights
* horizontal_flip=<font color=orange>True</font>
* vertical_flip=<font color=orange>True</font>

In [ ]:
def model_train(learning_rate):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True, #          [!]
        vertical_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 126s 720ms/step - AUC: 0.7232 - Precision: 0.8474 - Recall: 0.6630 - accuracy: 0.6639 - loss: 0.6158 - val_AUC: 0.9274 - val_Precision: 0.8927 - val_Recall: 0.8744 - val_accuracy: 0.8558 - val_loss: 0.3777
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 111s 680ms/step - AUC: 0.9463 - Precision: 0.9624 - Recall: 0.8565 - accuracy: 0.8673 - loss: 0.3027 - val_AUC: 0.9406 - val_Precision: 0.9140 - val_Recall: 0.8718 - val_accuracy: 0.8686 - val_loss: 0.3258
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 142s 683ms/step - AUC: 0.9645 - Precision: 0.9714 - Recall: 0.8744 - accuracy: 0.8887 - loss: 0.2485 - val_AUC: 0.9496 - val_Precision: 0.9321 - val_Recall: 0.8795 - val_accuracy: 0.8846 - val_loss: 0.2958
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 141s 679ms/step - AUC: 0.9700 - Precision: 0.9754 - Recall: 0.8962 - accuracy: 0.9046 - loss: 0.2177 - val_AUC: 0.9547 - val_Precision: 0.9372 - val_Recall: 0.8795 - val_accuracy: 0.8878 - val_loss: 0.2816
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_4.2'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

# cfg 5.0
* learning_rate=0.00005
* class_weights
* horizontal_flip=True
* Dense(<font color=orange>256</font>, ReLU)

In [ ]:
def model_train(learning_rate, dense_value):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]

    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005, dense_value=256)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 139s 790ms/step - AUC: 0.7717 - Precision: 0.8687 - Recall: 0.7239 - accuracy: 0.7123 - loss: 0.5632 - val_AUC: 0.9349 - val_Precision: 0.8642 - val_Recall: 0.8974 - val_accuracy: 0.8478 - val_loss: 0.3320
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 129s 731ms/step - AUC: 0.9567 - Precision: 0.9673 - Recall: 0.8796 - accuracy: 0.8870 - loss: 0.2629 - val_AUC: 0.9525 - val_Precision: 0.9206 - val_Recall: 0.8923 - val_accuracy: 0.8846 - val_loss: 0.2850
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 720ms/step - AUC: 0.9775 - Precision: 0.9842 - Recall: 0.8997 - accuracy: 0.9150 - loss: 0.1926 - val_AUC: 0.9629 - val_Precision: 0.8953 - val_Recall: 0.9205 - val_accuracy: 0.8830 - val_loss: 0.2512
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 122s 748ms/step - AUC: 0.9819 - Precision: 0.9819 - Recall: 0.9153 - accuracy: 0.9241 - loss: 0.1741 - val_AUC: 0.9652 - val_Precision: 0.9158 - val_Recall: 0.9205 - val_accuracy: 0.8974 - val_loss: 0.2382
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_5.0'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 5.1
* learning_rate=0.00005
* class_weights
* horizontal_flip=True
* Dense(<font color=orange><font color=orange>64</font></font>, ReLU)

In [ ]:
def model_train(learning_rate, dense_value):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005, dense_value=64)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │        81,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,340,033 (8.93 MB)

 Trainable params: 82,049 (320.50 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 135s 766ms/step - AUC: 0.6302 - Precision: 0.8052 - Recall: 0.7113 - accuracy: 0.6538 - loss: 0.7025 - val_AUC: 0.9236 - val_Precision: 0.8642 - val_Recall: 0.8974 - val_accuracy: 0.8478 - val_loss: 0.4372
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 727ms/step - AUC: 0.9338 - Precision: 0.9472 - Recall: 0.8591 - accuracy: 0.8586 - loss: 0.3712 - val_AUC: 0.9461 - val_Precision: 0.8834 - val_Recall: 0.9128 - val_accuracy: 0.8702 - val_loss: 0.3370
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 713ms/step - AUC: 0.9539 - Precision: 0.9660 - Recall: 0.8677 - accuracy: 0.8795 - loss: 0.2933 - val_AUC: 0.9530 - val_Precision: 0.8826 - val_Recall: 0.9256 - val_accuracy: 0.8766 - val_loss: 0.2965
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 719ms/step - AUC: 0.9654 - Precision: 0.9725 - Recall: 0.8875 - accuracy: 0.8977 - loss: 0.2475 - val_AUC: 0.9565 - val_Precision: 0.8922 - val_Recall: 0.9128 - val_accuracy: 0.8766 - val_loss: 0.2752
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_5.1'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

# cfg 6.0
* learning_rate=<font color=orange>0.00001</font>
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)

In [ ]:
def model_train(learning_rate, dense_value):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00001, dense_value=256)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 133s 756ms/step - AUC: 0.5305 - Precision: 0.7496 - Recall: 0.6775 - accuracy: 0.5925 - loss: 0.8326 - val_AUC: 0.7559 - val_Precision: 0.7216 - val_Recall: 0.7974 - val_accuracy: 0.6811 - val_loss: 0.5832
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 116s 713ms/step - AUC: 0.7917 - Precision: 0.8663 - Recall: 0.7514 - accuracy: 0.7282 - loss: 0.5465 - val_AUC: 0.8632 - val_Precision: 0.8519 - val_Recall: 0.8256 - val_accuracy: 0.8013 - val_loss: 0.4791
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 714ms/step - AUC: 0.9130 - Precision: 0.9369 - Recall: 0.8209 - accuracy: 0.8253 - loss: 0.3852 - val_AUC: 0.8942 - val_Precision: 0.8787 - val_Recall: 0.8359 - val_accuracy: 0.8253 - val_loss: 0.4244
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 718ms/step - AUC: 0.9360 - Precision: 0.9563 - Recall: 0.8339 - accuracy: 0.8488 - loss: 0.3349 - val_AUC: 0.9095 - val_Precision: 0.8838 - val_Recall: 0.8385 - val_accuracy: 0.8301 - val_loss: 0.3886
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_6.0'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 6.1
* <font color=orange>LearningRateScheduler</font>
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)

In [ ]:
def lr_scheduler(epoch, lr):
    if  0 <= epoch <= 2:
        return 0.00005
    else:
        return 0.00001

lr_callback = LearningRateScheduler(lr_scheduler)

In [ ]:
def model_train(lr_callback, dense_value):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights, #      [!]
        callbacks=[lr_callback]     #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(lr_callback, dense_value=256)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 134s 755ms/step - AUC: 0.7598 - Precision: 0.8576 - Recall: 0.7614 - accuracy: 0.7291 - loss: 0.6232 - val_AUC: 0.9320 - val_Precision: 0.8630 - val_Recall: 0.9205 - val_accuracy: 0.8590 - val_loss: 0.3387 - learning_rate: 5.0000e-05
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 719ms/step - AUC: 0.9610 - Precision: 0.9633 - Recall: 0.8922 - accuracy: 0.8942 - loss: 0.2533 - val_AUC: 0.9517 - val_Precision: 0.8878 - val_Recall: 0.9333 - val_accuracy: 0.8846 - val_loss: 0.2838 - learning_rate: 5.0000e-05
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 114s 702ms/step - AUC: 0.9726 - Precision: 0.9734 - Recall: 0.9014 - accuracy: 0.9084 - loss: 0.2101 - val_AUC: 0.9581 - val_Precision: 0.8941 - val_Recall: 0.9308 - val_accuracy: 0.8878 - val_loss: 0.2613 - learning_rate: 5.0000e-05
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 116s 711ms/step - AUC: 0.9783 - Precision: 0.9803 - Recall: 0.9170 - accuracy: 0.9245 - loss: 0.1852 - val_AUC: 0.9585 - val_Precision: 0.

In [ ]:
cfg = 'cfg_6.1'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 6.2
* <font color=orange>CosineDecay</font>
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)

In [ ]:
def model_train(dense_value):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # CosineDecay
    initial_learning_rate = 0.0001
    decay_steps = len(train_generator) * 10  # 10 эпох

    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate,
        decay_steps
        )

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=lr_schedule), # CosineDecay [!]
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(dense_value=256)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 129s 716ms/step - AUC: 0.8401 - Precision: 0.8987 - Recall: 0.7517 - accuracy: 0.7565 - loss: 0.4704 - val_AUC: 0.9519 - val_Precision: 0.8975 - val_Recall: 0.9205 - val_accuracy: 0.8846 - val_loss: 0.2805
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 664ms/step - AUC: 0.9779 - Precision: 0.9757 - Recall: 0.9052 - accuracy: 0.9135 - loss: 0.1954 - val_AUC: 0.9604 - val_Precision: 0.9093 - val_Recall: 0.9256 - val_accuracy: 0.8958 - val_loss: 0.2510
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 141s 660ms/step - AUC: 0.9832 - Precision: 0.9816 - Recall: 0.9182 - accuracy: 0.9262 - loss: 0.1653 - val_AUC: 0.9669 - val_Precision: 0.9144 - val_Recall: 0.9308 - val_accuracy: 0.9022 - val_loss: 0.2301
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 141s 658ms/step - AUC: 0.9866 - Precision: 0.9815 - Recall: 0.9278 - accuracy: 0.9335 - loss: 0.1487 - val_AUC: 0.9670 - val_Precision: 0.9188 - val_Recall: 0.9282 - val_accuracy: 0.9038 - val_loss: 0.2272
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_6.2'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

# cfg 7.0
* learning_rate=0.00005
* class_weights
* horizontal_flip=True
* Dense(256, ReLU) + <font color=orange>Dense(128, ReLU)</font>

In [ ]:
def model_train(learning_rate, dense_value_1, dense_value_2):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value_1, activation='relu'),
        layers.Dense(dense_value_2, activation='relu'), #      [!]
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005, dense_value_1=256, dense_value_2=128)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,618,945 (9.99 MB)

 Trainable params: 360,961 (1.38 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 124s 700ms/step - AUC: 0.7202 - Precision: 0.8500 - Recall: 0.6492 - accuracy: 0.6619 - loss: 0.6122 - val_AUC: 0.9408 - val_Precision: 0.8518 - val_Recall: 0.9282 - val_accuracy: 0.8542 - val_loss: 0.3308
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 107s 658ms/step - AUC: 0.9718 - Precision: 0.9754 - Recall: 0.9010 - accuracy: 0.9096 - loss: 0.2308 - val_AUC: 0.9561 - val_Precision: 0.8628 - val_Recall: 0.9513 - val_accuracy: 0.8750 - val_loss: 0.2796
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 660ms/step - AUC: 0.9810 - Precision: 0.9800 - Recall: 0.9208 - accuracy: 0.9273 - loss: 0.1784 - val_AUC: 0.9623 - val_Precision: 0.8765 - val_Recall: 0.9462 - val_accuracy: 0.8830 - val_loss: 0.2461
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 142s 659ms/step - AUC: 0.9846 - Precision: 0.9827 - Recall: 0.9265 - accuracy: 0.9329 - loss: 0.1594 - val_AUC: 0.9660 - val_Precision: 0.8523 - val_Recall: 0.9769 - val_accuracy: 0.8798 - val_loss: 0.2804
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_7.0'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 7.1
* learning_rate=0.00005
* class_weights
* horizontal_flip=True
* <font color=orange>Dense(128, ReLU) + Dense(64, ReLU)</font>

In [ ]:
def model_train(learning_rate, dense_value_1, dense_value_2):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value_1, activation='relu'),
        layers.Dense(dense_value_2, activation='relu'), #      [!]
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005, dense_value_1=128, dense_value_2=64)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,430,273 (9.27 MB)

 Trainable params: 172,289 (673.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 123s 694ms/step - AUC: 0.7088 - Precision: 0.8619 - Recall: 0.5385 - accuracy: 0.6019 - loss: 0.6321 - val_AUC: 0.9277 - val_Precision: 0.8954 - val_Recall: 0.8564 - val_accuracy: 0.8478 - val_loss: 0.3941
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 661ms/step - AUC: 0.9617 - Precision: 0.9691 - Recall: 0.8802 - accuracy: 0.8910 - loss: 0.2828 - val_AUC: 0.9494 - val_Precision: 0.8787 - val_Recall: 0.9103 - val_accuracy: 0.8654 - val_loss: 0.2958
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 660ms/step - AUC: 0.9750 - Precision: 0.9772 - Recall: 0.9088 - accuracy: 0.9156 - loss: 0.2108 - val_AUC: 0.9570 - val_Precision: 0.9257 - val_Recall: 0.8949 - val_accuracy: 0.8894 - val_loss: 0.2688
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 662ms/step - AUC: 0.9770 - Precision: 0.9787 - Recall: 0.9099 - accuracy: 0.9187 - loss: 0.1964 - val_AUC: 0.9643 - val_Precision: 0.8998 - val_Recall: 0.9436 - val_accuracy: 0.8990 - val_loss: 0.2431
Epoch 5/10
163/1

In [ ]:
cfg = 'cfg_7.1'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

# cfg 8.0
* learning_rate=0.00005
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)
* EPOCHS=<font color=orange>20</font> + EarlyStopping

In [ ]:
def model_train(learning_rate, dense_value, epochs=10):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # EarlyStopping       [!]
    callbacks_list = [
        EarlyStopping(
            monitor="val_AUC",
            patience=3, ),
        ModelCheckpoint(
            filepath="/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-8-0_checkpoint.keras",
            monitor="val_loss",
            save_best_only=True
        )
    ]

    # Обучение модели
    # EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=epochs,  #      [!]
        validation_data=val_generator,
        class_weight=class_weights, #      [!]
        callbacks=callbacks_list    #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005, dense_value=256, epochs=20)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 132s 735ms/step - AUC: 0.7340 - Precision: 0.8461 - Recall: 0.6543 - accuracy: 0.6604 - loss: 0.6131 - val_AUC: 0.9238 - val_Precision: 0.8582 - val_Recall: 0.8846 - val_accuracy: 0.8365 - val_loss: 0.3521
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 718ms/step - AUC: 0.9618 - Precision: 0.9663 - Recall: 0.8742 - accuracy: 0.8847 - loss: 0.2558 - val_AUC: 0.9446 - val_Precision: 0.8825 - val_Recall: 0.9051 - val_accuracy: 0.8654 - val_loss: 0.2973
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 114s 698ms/step - AUC: 0.9722 - Precision: 0.9749 - Recall: 0.8987 - accuracy: 0.9086 - loss: 0.2153 - val_AUC: 0.9561 - val_Precision: 0.9038 - val_Recall: 0.9154 - val_accuracy: 0.8862 - val_loss: 0.2656
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 717ms/step - AUC: 0.9808 - Precision: 0.9796 - Recall: 0.9132 - accuracy: 0.9209 - loss: 0.1788 - val_AUC: 0.9612 - val_Precision: 0.9066 - val_Recall: 0.9205 - val_accuracy: 0.8910 - val_loss: 0.2481
Epoch 5/20
163/1

In [ ]:
cfg = 'cfg_8.0'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 8.1
* learning_rate=<font color=orange>0.00001</font>
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)
* EPOCHS=<font color=orange>20</font> + EarlyStopping

In [ ]:
def model_train(learning_rate, dense_value, epochs=10):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # EarlyStopping       [!]
    callbacks_list = [
        EarlyStopping(
            monitor="val_AUC",
            patience=3, ),
        ModelCheckpoint(
            filepath="/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-8-1_checkpoint.keras",
            monitor="val_loss",
            save_best_only=True
        )
    ]

    # Обучение модели
    # EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=epochs,  #      [!]
        validation_data=val_generator,
        class_weight=class_weights, #      [!]
        callbacks=callbacks_list    #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00001, dense_value=256, epochs=20)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 129s 738ms/step - AUC: 0.5296 - Precision: 0.7612 - Recall: 0.4010 - accuracy: 0.4583 - loss: 0.8177 - val_AUC: 0.8838 - val_Precision: 0.8723 - val_Recall: 0.8231 - val_accuracy: 0.8141 - val_loss: 0.5129
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 112s 686ms/step - AUC: 0.8009 - Precision: 0.8940 - Recall: 0.7375 - accuracy: 0.7383 - loss: 0.5377 - val_AUC: 0.9171 - val_Precision: 0.9003 - val_Recall: 0.8333 - val_accuracy: 0.8381 - val_loss: 0.4320
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 692ms/step - AUC: 0.9001 - Precision: 0.9360 - Recall: 0.8090 - accuracy: 0.8180 - loss: 0.4146 - val_AUC: 0.9270 - val_Precision: 0.9059 - val_Recall: 0.8641 - val_accuracy: 0.8590 - val_loss: 0.3845
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 693ms/step - AUC: 0.9312 - Precision: 0.9510 - Recall: 0.8470 - accuracy: 0.8530 - loss: 0.3450 - val_AUC: 0.9343 - val_Precision: 0.8950 - val_Recall: 0.8744 - val_accuracy: 0.8574 - val_loss: 0.3523
Epoch 5/20
163/1

In [ ]:
cfg = 'cfg_8.1'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 8.2
* learning_rate=<font color=orange>0.00001</font>
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)
* EPOCHS=<font color=orange>20</font> + EarlyStopping
* base_model.trainable = True

In [ ]:
def model_train(learning_rate, dense_value, epochs=10):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = True  #       [!]

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # EarlyStopping       [!]
    callbacks_list = [
        EarlyStopping(
            monitor="val_AUC",
            patience=5, ), # [!]
        ModelCheckpoint(
            filepath="/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-8-2_checkpoint.keras",
            monitor="val_loss",
            save_best_only=True
        )
    ]

    # Обучение модели
    # EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=epochs,  #      [!]
        validation_data=val_generator,
        class_weight=class_weights, #      [!]
        callbacks=callbacks_list    #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00001, dense_value=256, epochs=20)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 2,552,065 (9.74 MB)

 Non-trainable params: 34,112 (133.25 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 170s 768ms/step - AUC: 0.7797 - Precision: 0.8607 - Recall: 0.7413 - accuracy: 0.7210 - loss: 0.5573 - val_AUC: 0.8298 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 0.8662
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 179s 745ms/step - AUC: 0.9698 - Precision: 0.9736 - Recall: 0.9071 - accuracy: 0.9135 - loss: 0.2362 - val_AUC: 0.8672 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 1.0538
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 720ms/step - AUC: 0.9839 - Precision: 0.9869 - Recall: 0.9307 - accuracy: 0.9387 - loss: 0.1610 - val_AUC: 0.9021 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 1.1063
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 719ms/step - AUC: 0.9901 - Precision: 0.9881 - Recall: 0.9401 - accuracy: 0.9468 - loss: 0.1279 - val_AUC: 0.9050 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 1.2023
Epoch 5/20
163/1

In [ ]:
cfg = 'cfg_8.2'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 8.3
* <font color=orange>LearningRateScheduler</font>
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)
* base_model.trainable = True

In [ ]:
def lr_scheduler(epoch, lr):
    if  0 <= epoch <= 2:
        return 0.0001
    else:
        return 0.00001

lr_callback = LearningRateScheduler(lr_scheduler)

In [ ]:
def model_train(lr_callback, dense_value, epochs=10):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # Аугментация
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True #             [!]
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Генераторы
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='rgb'
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False,
        color_mode='rgb'
    )

    print()

    # class_weights      [!]
    class_weights = compute_class_weight('balanced',
                                         classes=np.unique(train_generator.classes),
                                         y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = True  #   [!]

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])


    # Компиляция модели
    model.compile(
        optimizer=Adam(),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()


    # Обучение модели

    start = time.time() # Фиксация времени

    history = model.fit(
        train_generator,
        epochs=epochs,  #      [!]
        validation_data=val_generator,
        class_weight=class_weights, #      [!]
        callbacks=[lr_callback]    #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(lr_callback, dense_value=256, epochs=20)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

{0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}



Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 2,552,065 (9.74 MB)

 Non-trainable params: 34,112 (133.25 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 170s 755ms/step - AUC: 0.9445 - Precision: 0.9666 - Recall: 0.8568 - accuracy: 0.8738 - loss: 0.2733 - val_AUC: 0.8647 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 1.4932 - learning_rate: 1.0000e-04
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 723ms/step - AUC: 0.9949 - Precision: 0.9906 - Recall: 0.9615 - accuracy: 0.9645 - loss: 0.0853 - val_AUC: 0.5000 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 3.3564 - learning_rate: 1.0000e-04
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 716ms/step - AUC: 0.9967 - Precision: 0.9942 - Recall: 0.9773 - accuracy: 0.9789 - loss: 0.0594 - val_AUC: 0.5021 - val_Precision: 0.6250 - val_Recall: 1.0000 - val_accuracy: 0.6250 - val_loss: 4.6554 - learning_rate: 1.0000e-04
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 719ms/step - AUC: 0.9979 - Precision: 0.9970 - Recall: 0.9730 - accuracy: 0.9777 - loss: 0.0553 - val_AUC: 0.5278 - val_Precision: 0.

In [ ]:
cfg = 'cfg_8.3'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

# cfg 9.0
* learning_rate=0.00005
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)
* <font color=orange>bicubic</font> `base_path = '/content/content/chest_xray_bicubic'`

In [ ]:
def model_train(learning_rate, dense_value, epochs=10):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # 1: Загрузка данных с interpolation='bicubic' #      [!]
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        label_mode='binary',
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        color_mode='rgb',
        interpolation='bicubic'
    )

    val_dataset = tf.keras.utils.image_dataset_from_directory(
        val_dir,
        label_mode='binary',
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        color_mode='rgb',
        interpolation='bicubic'
    )

    test_dataset = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        label_mode='binary',
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        color_mode='rgb',
        interpolation='bicubic'
    )

    # 2: Аугментация и нормализация (rescale) #      [!]
    data_augmentation = tf.keras.Sequential([
        layers.Rescaling(1./255),
        layers.RandomRotation(0.03),         # ~10 градусов
        layers.RandomZoom(0.1),
        layers.RandomBrightness(0.1),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomFlip("horizontal")
    ])

    # 3: Подготовка финальных датасетов #      [!]
    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = (
        train_dataset
        .shuffle(1000)
        .map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
        .prefetch(AUTOTUNE)
    )

    val_ds = val_dataset.map(lambda x, y: (x / 255.0, y)).prefetch(AUTOTUNE)
    test_ds = test_dataset.map(lambda x, y: (x / 255.0, y)).prefetch(AUTOTUNE)

    print()

    # class_weights      [!]
    class_weights = {
        0: 1.9448173005219984,   # для класса "NORMAL"
        1: 0.6730322580645162    # для класса "PNEUMONIA"
        }
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    #EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_ds, #      [!]
        epochs=epochs,
        validation_data=val_ds, #      [!]
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.00005, dense_value=256, epochs=20)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.

{0: 1.9448173005219984, 1: 0.6730322580645162}



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)


Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 422ms/step - AUC: 0.5029 - Precision: 0.7430 - Recall: 0.5088 - accuracy: 0.5047 - loss: 0.8084 - val_AUC: 0.7476 - val_Precision: 0.6625 - val_Recall: 0.9462 - val_accuracy: 0.6651 - val_loss: 0.5953
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 132s 403ms/step - AUC: 0.5514 - Precision: 0.7799 - Recall: 0.5757 - accuracy: 0.5580 - loss: 0.7162 - val_AUC: 0.8288 - val_Precision: 0.9639 - val_Recall: 0.4103 - val_accuracy: 0.6218 - val_loss: 0.6428
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 102s 390ms/step - AUC: 0.5492 - Precision: 0.7660 - Recall: 0.5284 - accuracy: 0.5319 - loss: 0.7134 - val_AUC: 0.8657 - val_Precision: 0.7592 - val_Recall: 0.8974 - val_accuracy: 0.7580 - val_loss: 0.5177
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 103s 403ms/step - AUC: 0.5999 - Precision: 0.7880 - Recall: 0.6031 - accuracy: 0.5845 - loss: 0.6758 - val_AUC: 0.8762 - val_Precision: 0.7972 - val_Recall: 0.8667 - val_accuracy: 0.7788 - val_loss: 0.5042
Epoch 5/20
163/

In [ ]:
cfg = 'cfg_9.0'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")

## cfg 9.1
* learning_rate=0.0001
* class_weights
* horizontal_flip=True
* Dense(256, ReLU)
* <font color=orange>bicubic</font> `image_dataset_from_directory`

In [ ]:
def model_train(learning_rate, dense_value, epochs=10):
    # Размер изображений и параметры
    IMG_SIZE = (224, 224)  # MobileNetV2 лучше работает с этим размером
    BATCH_SIZE = 32

    # 1: Загрузка данных с interpolation='bicubic' #      [!]
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        label_mode='binary',
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        color_mode='rgb',
        interpolation='bicubic'
    )

    val_dataset = tf.keras.utils.image_dataset_from_directory(
        val_dir,
        label_mode='binary',
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        color_mode='rgb',
        interpolation='bicubic'
    )

    test_dataset = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        label_mode='binary',
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        color_mode='rgb',
        interpolation='bicubic'
    )

    # 2: Аугментация и нормализация (rescale) #      [!]
    data_augmentation = tf.keras.Sequential([
        layers.Rescaling(1./255),
        layers.RandomRotation(0.03),         # ~10 градусов
        layers.RandomZoom(0.1),
        layers.RandomBrightness(0.1),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomFlip("horizontal")
    ])

    # 3: Подготовка финальных датасетов #      [!]
    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = (
        train_dataset
        .shuffle(1000)
        .map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
        .prefetch(AUTOTUNE)
    )

    val_ds = val_dataset.map(lambda x, y: (x / 255.0, y)).prefetch(AUTOTUNE)
    test_ds = test_dataset.map(lambda x, y: (x / 255.0, y)).prefetch(AUTOTUNE)

    print()

    # class_weights      [!]
    class_weights = {
        0: 1.9448173005219984,   # для класса "NORMAL"
        1: 0.6730322580645162    # для класса "PNEUMONIA"
        }
    print(class_weights)

    print()

    # Загрузка MobileNetV2 без верхушки
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Фиксируем веса

    # Добавляем слои классификации
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(dense_value, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'AUC',
            'Recall',
            'Precision',
            'accuracy'
        ]
    )

    # Отображение информации об устройстве сети:
    model.summary()

    print()

    # Обучение модели
    #EPOCHS = 10

    start = time.time() # Фиксация времени

    history = model.fit(
        train_ds, #      [!]
        epochs=epochs,
        validation_data=val_ds, #      [!]
        class_weight=class_weights #      [!]
    )

    end = time.time() # Фиксация времени

    model_history = history.history
    model_history['Wall time'] = end - start

    print()

    for k, v in model_history.items():
        print(k, v)

    return model, model_history

In [ ]:
model, model_history = model_train(learning_rate=0.0001, dense_value=256, epochs=20)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.

{0: 1.9448173005219984, 1: 0.6730322580645162}



Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)


Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 120s 434ms/step - AUC: 0.5193 - Precision: 0.7695 - Recall: 0.5557 - accuracy: 0.5357 - loss: 0.7503 - val_AUC: 0.8378 - val_Precision: 0.7818 - val_Recall: 0.8359 - val_accuracy: 0.7516 - val_loss: 0.5416
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 102s 394ms/step - AUC: 0.5917 - Precision: 0.7884 - Recall: 0.5387 - accuracy: 0.5523 - loss: 0.6853 - val_AUC: 0.8882 - val_Precision: 0.8491 - val_Recall: 0.8077 - val_accuracy: 0.7901 - val_loss: 0.4576
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 143s 403ms/step - AUC: 0.6358 - Precision: 0.8162 - Recall: 0.5647 - accuracy: 0.5853 - loss: 0.6647 - val_AUC: 0.9164 - val_Precision: 0.9491 - val_Recall: 0.6692 - val_accuracy: 0.7708 - val_loss: 0.4741
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 103s 399ms/step - AUC: 0.6635 - Precision: 0.8129 - Recall: 0.6104 - accuracy: 0.6058 - loss: 0.6452 - val_AUC: 0.9129 - val_Precision: 0.8808 - val_Recall: 0.8333 - val_accuracy: 0.8253 - val_loss: 0.4039
Epoch 5/20
163/

In [ ]:
cfg = 'cfg_9.1'
add_result(cfg, model_history)
model.save(f"/content/drive/MyDrive/DS/chest-xray-pneumonia/MobileNetV2-{cfg}.keras")